In [2]:
!pip install -q https://github.com/buguroo/pyknow/archive/v1.7.0.tar.gz

In [3]:
!pip install yfinance==0.1.80

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import collections
import collections.abc

if not hasattr(collections, 'Mapping'):
    collections.Mapping=collections.abc.Mapping

from pyknow import *
import random

In [6]:
import random
from pyknow import *


class WinTotals(Fact):
    human = Field(int, default=0)
    computer = Field(int, default=0)
    ties = Field(int, default=0)


class TicTacToeRules(Fact):
    winner = Field(str, mandatory=True)
    combination = Field(list, mandatory=True)


class ValidMove(Fact):
    move = Field(str, mandatory=True)
    key = Field(str, mandatory=True)


class Action(Fact):
    pass


class HumanChoice(Fact):
    pass


class ComputerChoice(Fact):
    pass


class TicTacToe(KnowledgeEngine):

    @DefFacts()
    def game_setup(self):
        """Declare game rules and valid input keys for the user."""
        print('Setting up TicTacToe game rules...')

        self.valid_answers = dict()
        for i in range(9, 0, -1):  # 1 to 9
            self.declare(ValidMove(move='row %d, column %d' % ((i - 1) // 3, (i - 1) % 3), key=str(i)))

        # Define rules for winning combinations
        yield TicTacToeRules(winner='X', combination=[[0, 1, 2], [3, 4, 5], [6, 7, 8],
                                                       [0, 3, 6], [1, 4, 7], [2, 5, 8],
                                                       [0, 4, 8], [2, 4, 6]])

        print('All game rules in place.')

    @Rule()
    def startup(self):
        print("Let's play a game of TicTacToe!")
        print("You are X, I am O")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))

    @Rule(NOT(Action()),
          ValidMove(move=MATCH.move,
                    key=MATCH.key))
    def store_valid_answers(self, move, key):
        print('store_valid_answers', key, move)
        self.valid_answers[key] = move

    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        print("Getting human move...")
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        print("Human move:", res)
        self.declare(HumanChoice(res))

    @Rule(AS.f1 << HumanChoice(MATCH.choice),
        ValidMove(move=MATCH.move,
                    key=MATCH.choice),
        AS.f2 << Action('get-human-move'))
    def good_human_move(self, f1, f2, move):
        self.retract(f1)
        self.retract(f2)
        self.declare(HumanChoice(move))
        # Debugging print statements
        # print("Before deleting:", self.valid_answers)
        # row = move.split(',')[0].strip()
        # print("Row extracted from move:", row)
        # print("Deleting row:", row)
        # deleted_move = self.valid_answers.pop(row, None)
        # if deleted_move is not None:
            # print("After deleting:", self.valid_answers)
        self.declare(Action('get-computer-move'))
        # else:
        #     print("Key does not exist:", row)



    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          NOT(ValidMove(key=MATCH.choice)),
          AS.f2 << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print("Sorry %s is not a valid answer" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))

    #
    # COMPUTER MOVE RULES
    #
    @Rule(AS.f1 << Action('get-computer-move'))
    def get_computer_move(self, f1):
        print("Getting computer move...")
        # Get the list of valid moves not chosen by the human player
        valid_moves = [move for key, move in self.valid_answers.items() if key not in self.facts.keys()]
        # Choose a move randomly from the remaining valid moves
        choice = random.choice(valid_moves)
        print("Computer move:", choice)
        self.retract(f1)
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))
        
    #
    # WIN DETERMINATION RULES
    #
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.w << WinTotals(computer=MATCH.cw),
          TicTacToeRules(winner=MATCH.cc))
    def computer_wins(self, f1, f2, f3, w, cw):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, computer=cw + 1)
        print("Computer wins!")
        self.declare(Action('determine-play-again'))

    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          'w' << WinTotals(human=MATCH.hw),
          TicTacToeRules(winner=MATCH.hc))
    def humans_wins(self, f1, f2, f3, w, hw):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, human=hw + 1)
        print("You win!")
        self.declare(Action('determine-play-again'))

    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.cc),
          AS.w << WinTotals(ties=MATCH.nt))
    def tie(self, f1, f2, f3, w, nt):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, ties=nt + 1)
        print("Tie! Ha-ha!")
        self.declare(Action('determine-play-again'))

    #
    # PLAY AGAIN RULE
    #
    @Rule(AS.f1 << Action('determine-play-again'),
          WinTotals(computer=MATCH.ct,
                    human=MATCH.ht,
                    ties=MATCH.tt))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1)
        if not self.yes_or_no("Play again? Y/N"):
            print("You won", ht, "game(s).")
            print("Computer won", ct, "game(s).")
            print("We tied", tt, "game(s).")
            self.halt()
        else:
            self.declare(Action('get-human-move'))

    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')
    

    @Rule(AS.f1 << Action('determine-results'),
          WinTotals(computer=MATCH.cw),
          TEST(lambda cw: cw < 3))
    def check_continue_game_computer(self, f1, cw):
        # Check if the game should continue
        if cw < 3:  # Change condition based on your game end criteria
            self.declare(Action('get-human-move'))
        else:
            print("Game over! Computer has won 3 games.")
            self.halt()

    @Rule(AS.f1 << Action('determine-results'),
          WinTotals(ties=MATCH.nt),
          TEST(lambda nt: nt < 3))
    def check_continue_game_tie(self, f1, nt):
        # Check if the game should continue due to tie
        if nt < 3:  # Change condition based on your game end criteria
            self.declare(Action('get-human-move'))
        else:
            print("Game over! It's a tie 3 times.")
            self.halt()

    @Rule(AS.f1 << Action('determine-results'),
          WinTotals(human=MATCH.hw),
          TEST(lambda hw: hw < 3))
    def check_continue_game_human(self, f1, hw):
        # Check if the game should continue
        if hw < 3:  # Change condition based on your game end criteria
            self.declare(Action('get-computer-move'))
        else:
            print("Game over! You have won 3 games.")
            self.halt()

    @Rule(AS.f1 << Action('determine-results'),
          WinTotals(computer=MATCH.cw),
          TicTacToeRules(winner='X', combination=['X', 'X', 'X']))
    def check_continue_game_win(self, f1, cw):
        # Check if the game should continue due to human win
        if cw < 3:  # Change condition based on your game end criteria
            self.declare(Action('get-human-move'))
        else:
            print("Game over! Computer has won 3 games.")
            self.halt()


game = TicTacToe()
game.reset()
game.run()


Setting up TicTacToe game rules...
All game rules in place.
store_valid_answers 1 row 0, column 0
store_valid_answers 2 row 0, column 1
store_valid_answers 3 row 0, column 2
store_valid_answers 4 row 1, column 0
store_valid_answers 5 row 1, column 1
store_valid_answers 6 row 1, column 2
store_valid_answers 7 row 2, column 0
store_valid_answers 8 row 2, column 1
store_valid_answers 9 row 2, column 2
Let's play a game of TicTacToe!
You are X, I am O
Getting human move...


# test


In [64]:
from pyknow import KnowledgeEngine, Rule, MATCH, Fact
import random

class TicTacToeState(Fact):
    pass

class TicTacToe(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def reset(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def display_board(self):
        for row in self.board:
            print(" {} | {} | {} ".format(*row))
            print("---|---|---")

    def make_move(self, player, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        if self.board[row][col] == " ":
            self.board[row][col] = player
            self.remaining_moves.remove(position)
            if self.check_win(player):
                self.game_over = True
                print(f"Player {player} wins!")
                
        else:
            print("Invalid move! Position already taken.")

    def check_win(self, player):
        # Check rows
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Check columns
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Check diagonals
        if all(self.board[i][i] == player for i in range(3)) or all(self.board[i][2 - i] == player for i in range(3)):
            return True
        return False

class TicTacToePlayer(KnowledgeEngine):
    def __init__(self, board):
        super().__init__()
        self.board = board
        self.computer_move = None  # Store the computer's move

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves))
    def get_computer_move(self, remaining_moves):
        valid_moves = [move for move in remaining_moves if self.is_valid(move)]
        position = random.choice(valid_moves)
        self.computer_move = position  # Store the selected position

    def is_valid(self, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        return self.board[row][col] == " "

def play_game():
    game = TicTacToe()
    print("Game Start")
    
    while not game.game_over:
        print("Human's Turn :")
        game.display_board()
        human_move = input("Enter your move (1-9): ")
        
        game.make_move("X", int(human_move))
        print("Human chose position", human_move)
        if game.remaining_moves and not game.game_over:
            # print("---------------")
            computer_player = TicTacToePlayer(game.board)
            computer_player.reset()
            computer_player.declare(TicTacToeState(remaining_moves=game.remaining_moves))
            computer_player.get_computer_move(game.remaining_moves)
            computer_player.run()

            # Access the computer player's facts
            # computer_facts = computer_player.facts
            # print("Computer player facts:", computer_facts)

            if computer_player.computer_move:
                position = computer_player.computer_move
                print(f"Computer chose position {position}.")
                game.make_move("O", position)
                if game.game_over:
                    game.display_board()
                    print("Computer wins!")
            else:
                print("No move made by the computer!")

               #game.make_move("O", int(computer_move))
        elif not game.game_over:
            print("It's a tie!")
            break

    game.display_board()

if __name__ == "__main__":
    play_game()


Game Start
Human's Turn :
   |   |   
---|---|---
   |   |   
---|---|---
   |   |   
---|---|---
Human chose position 5
Computer chose position 9.
Human's Turn :
   |   |   
---|---|---
   | X |   
---|---|---
   |   | O 
---|---|---
Invalid move! Position already taken.
Human chose position 9
Computer chose position 7.
Human's Turn :
   |   |   
---|---|---
   | X |   
---|---|---
 O |   | O 
---|---|---
Human chose position 1
Computer chose position 3.
Human's Turn :
 X |   | O 
---|---|---
   | X |   
---|---|---
 O |   | O 
---|---|---
Human chose position 4
Computer chose position 2.
Human's Turn :
 X | O | O 
---|---|---
 X | X |   
---|---|---
 O |   | O 
---|---|---
Human chose position 8
Computer chose position 6.
Player O wins!
 X | O | O 
---|---|---
 X | X | O 
---|---|---
 O | X | O 
---|---|---
Computer wins!
 X | O | O 
---|---|---
 X | X | O 
---|---|---
 O | X | O 
---|---|---


## advanced test

In [71]:
from pyknow import KnowledgeEngine, Rule, MATCH, Fact
import random

class TicTacToeState(Fact):
    pass

class TicTacToe(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def reset(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def display_board(self):
        for row in self.board:
            print(" {} | {} | {} ".format(*row))
            print("---|---|---")


    def make_move(self, player, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        if self.board[row][col] == " ":
            self.board[row][col] = player
            self.remaining_moves.remove(position)
            if self.check_win(player):
                self.game_over = True
                print(f"Player {player} wins!")                
        else:
            print("Invalid move! Position already taken.")
            return False  # Indicate that the move was invalid
        return True  # Indicate that the move was valid


    def check_win(self, player):
        # Check rows
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Check columns
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Check diagonals
        if all(self.board[i][i] == player for i in range(3)) or all(self.board[i][2 - i] == player for i in range(3)):
            return True
        return False

class TicTacToePlayer(KnowledgeEngine):
    def __init__(self, board):
        super().__init__()
        self.board = board
        self.computer_move = None  # Store the computer's move

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves))
    def get_computer_move(self, remaining_moves):
        valid_moves = [move for move in remaining_moves if self.is_valid(move)]
        position = random.choice(valid_moves)
        self.computer_move = position  # Store the selected position

    def is_valid(self, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        return self.board[row][col] == " "

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_corner(self):
        if 5 in self.board:
            self.computer_move = 5
        else:
            corners = [1, 3, 7, 9]
            valid_corners = [corner for corner in corners if self.is_valid(corner)]
            if valid_corners:
                self.computer_move = random.choice(valid_corners)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={5})))
    def choose_center(self):
        if self.is_valid(5):
            self.computer_move = 5

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={1, 3, 7, 9})))
    def prioritize_edges(self):
        edges = [2, 4, 6, 8]
        valid_edges = [edge for edge in edges if self.is_valid(edge)]
        if valid_edges:
            self.computer_move = random.choice(valid_edges)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), TicTacToeState(remaining_moves={1, 3, 7, 9}), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_diagonals(self):
        diagonals = [1, 3, 7, 9]
        valid_diagonals = [diagonal for diagonal in diagonals if self.is_valid(diagonal)]
        if valid_diagonals:
            self.computer_move = random.choice(valid_diagonals)

def play_game():
    game = TicTacToe()
    print("Game Start")
    
    # while not game.game_over:
    #     print("Human's Turn :")
    #     game.display_board()
    #     human_move = input("Enter your move (1-9): ")
        
    #     game.make_move("X", int(human_move))
    #     print("Human chose position", human_move)
        
    while not game.game_over:
        print("Human's Turn :")
        game.display_board()
        human_move = input("Enter your move (1-9): ")
        
        if human_move.isdigit() and int(human_move) in game.remaining_moves:
            if game.make_move("X", int(human_move)):
                print("Human chose position", human_move)
            else:
                continue  # Prompt user to enter move again if it's invalid
        else:
            print("Invalid move! Please enter a valid position (1-9) that is not already taken.")
            continue  # Prompt user to enter move again if it's invalid

        if game.remaining_moves and not game.game_over:
            computer_player = TicTacToePlayer(game.board)
            computer_player.reset()
            computer_player.declare(TicTacToeState(remaining_moves=game.remaining_moves))
            computer_player.get_computer_move(game.remaining_moves)
            computer_player.run()

            if computer_player.computer_move:
                position = computer_player.computer_move
                print(f"Computer chose position {position}.")
                game.make_move("O", position)
                if game.game_over:
                    game.display_board()
                    print("Computer wins!")
            else:
                print("No move made by the computer!")

        if not game.game_over and not game.remaining_moves:
            print("It's a tie!")
            break

    game.display_board()

if __name__ == "__main__":
    play_game()


Game Start
Human's Turn :
   |   |   
---|---|---
   |   |   
---|---|---
   |   |   
---|---|---
Human chose position 1
Computer chose position 5.
Human's Turn :
 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   
---|---|---
Invalid move! Please enter a valid position (1-9) that is not already taken.
Human's Turn :
 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   
---|---|---
Human chose position 2
Computer chose position 8.
Human's Turn :
 X | X |   
---|---|---
   | O |   
---|---|---
   | O |   
---|---|---
Invalid move! Please enter a valid position (1-9) that is not already taken.
Human's Turn :
 X | X |   
---|---|---
   | O |   
---|---|---
   | O |   
---|---|---
Player X wins!
Human chose position 3
 X | X | X 
---|---|---
   | O |   
---|---|---
   | O |   
---|---|---
